In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from datetime import datetime
import numpy as np
from IPython.display import Image, display

In [ ]:
%%bash

JULIA_VERSION="1.4.2"
JULIA_PACKAGES="DataFrames Feather ShiftedArrays RollingFunctions CSV Pipe Plots Flux Images Statistics"

JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
wget -nv $URL -O /tmp/julia.tar.gz
tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
rm /tmp/julia.tar.gz
for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'"'
done

In [ ]:

import os
os.environ['JULIA_NUM_THREADS'] = "4"

In [ ]:
%%time
!pip install --quiet julia
import julia
from julia.api import Julia
julia.install()
jl = Julia(compiled_modules=False)  # cannot use precompiled packages with pyjulia on linux :-(
%load_ext julia.magic

In [ ]:
%%julia
using Pkg
Pkg.add("PyCall")

In [ ]:
%%julia
using PyCall
ImageDataGenerator = pyimport("keras").preprocessing.image.ImageDataGenerator
train = ImageDataGenerator(rescale = 1/255)
train_dir = "../input/dogs-cats-images/dataset/training_set/"
train_generator = train.flow_from_directory(train_dir, target_size = (224, 224), batch_size = 20, class_mode = "binary")

In [ ]:
%%julia
using PyCall
ImageDataGenerator = pyimport("keras").preprocessing.image.ImageDataGenerator
train = ImageDataGenerator(rescale = 1/255)
train_dir = "../input/dogs-cats-images/dataset/test_set"
train_generator = train.flow_from_directory(train_dir, target_size = (224, 224), batch_size = 20, class_mode = "binary")

In [ ]:
%%julia
using Flux
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
end

In [ ]:
%%julia
img,label = train_generator
print(label)